In [1]:
# Run this cell to import the packages you will need to unpack the dataset
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pickle
import os
import zipfile
import scipy.ndimage
import tensorflow as tf

import sys
import json
import codecs

## Getting the Data
* Using validation as training

In [2]:
## loading the current arrays

train_q = np.load("val_qs_6_1.npy")
train_q2 = np.load("val_qs_6_2.npy")
train_q3 = np.load("val_qs_6_5.npy")

train_ans = np.load("val_ans_6_1.npy")
train_ans2 = np.load("val_ans_6_2.npy")
train_ans3 = np.load("val_ans_6_5.npy")

In [3]:
train_ims = np.load("val_ims_6_1.npy")
train_ims2 = np.load("val_ims_6_2.npy")
train_ims3 = np.load("val_ims_6_5.npy")

In [4]:
num_food_types = 36
num_ans = train_ans.shape[1]

745

In [4]:
## splitting half the data for train
combined_train_q = np.zeros(train_q.shape)
combined_train_ans = np.zeros(train_ans.shape)
combined_train_ims = []

for i in range(len(train_q)):
    if i%3==0:
        combined_train_q[i] = train_q[i]
        combined_train_ans[i]=train_ans[i]
        combined_train_ims.append(train_ims[i])
    elif i%3==1:
        combined_train_q[i] = train_q2[i]
        combined_train_ans[i] = train_ans2[i]
        combined_train_ims.append(train_ims2[i])
    else:
        combined_train_q[i] = train_q3[i]
        combined_train_ans[i] = train_ans3[i]
        combined_train_ims.append(train_ims3[i])

In [8]:
len(combined_train_ims)
combined_train_ims[744].shape

(224, 224, 3)

## Model

In [5]:
import keras
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
from keras import models
from keras import layers
from keras import optimizers

## when adding new answers
k = 0

#Load the VGG model
vgg_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape = (224, 224, 3))
#inception_model = inception_v3.InceptionV3(weights='imagenet')
# Freeze the layers except the last 4 layers
for layer in vgg_model.layers[:-4]:
    layer.trainable = False
## Check the trainable status of the individual layers
for layer in vgg_model.layers:
    print(layer, layer.trainable)

# Create the IMAGE model
img_layer1 = layers.Flatten()(vgg_model.output)
img_layer2 = layers.Dense(1024, activation='relu')(img_layer1)
img_layer3 = layers.Dropout(0.5)(img_layer2)
img_layer4 = layers.Dense(num_food_types, activation='relu')(img_layer3)

# Create the QUESTION model
q_layer1 = layers.Input((train_q.shape[1],))
q_layer2 = layers.Embedding(2000, 100, input_length=50)(q_layer1)
q_layer3 = layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2)(q_layer2)
q_layer4 = layers.Dense(50, activation = 'relu')(q_layer3)

# Add layers to help merge the image and question
merged = layers.concatenate([img_layer4, q_layer4])
fc_1 = layers.Dense(100, activation = 'relu')(merged)
output = layers.Dense(num_ans + k, activation = 'softmax')(fc_1)
model = models.Model([vgg_model.input, q_layer1], output)
# Add layers to output
#model.add(layers.Dense())
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Using TensorFlow backend.


<keras.engine.input_layer.InputLayer object at 0x7fee7222cf60> False
<keras.layers.convolutional.Conv2D object at 0x7fee720adc18> False
<keras.layers.convolutional.Conv2D object at 0x7fee72059358> False
<keras.layers.pooling.MaxPooling2D object at 0x7fee72059d68> False
<keras.layers.convolutional.Conv2D object at 0x7fee72087f60> False
<keras.layers.convolutional.Conv2D object at 0x7fee71823940> False
<keras.layers.pooling.MaxPooling2D object at 0x7fee71836898> False
<keras.layers.convolutional.Conv2D object at 0x7fee717c5828> False
<keras.layers.convolutional.Conv2D object at 0x7fee71814940> False
<keras.layers.convolutional.Conv2D object at 0x7fee717eef28> False
<keras.layers.pooling.MaxPooling2D object at 0x7fee71794940> False
<keras.layers.convolutional.Conv2D object at 0x7fee717a18d0> False
<keras.layers.convolutional.Conv2D object at 0x7fee717b79e8> False
<keras.layers.convolutional.Conv2D object at 0x7fee7174dfd0> False
<keras.layers.pooling.MaxPooling2D object at 0x7fee717729e8>

## Training

In [20]:
train_ans

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
model.fit([combined_train_ims, combined_train_q], combined_train_ans, epochs = 6, batch_size = 30)

Epoch 1/6
 60/745 [=>............................] - ETA: 4:03 - loss: 0.6983 - acc: 0.8000

InvalidArgumentError: indices[25,49] = -2147483648 is not in [0, 2000)
	 [[Node: embedding_1/embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training/RMSprop/Assign_17"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_1/embeddings/read, embedding_1/Cast, training/RMSprop/gradients/embedding_1/embedding_lookup_grad/concat/axis)]]

In [6]:
i = 0
model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

for j in range(20):
    t_ims = None
    t_qs = None
    t_ans = None
    if j%2==0:
        t_ims = train_ims
        t_qs = train_q
        t_ans = train_ans
    else:
        t_ims = train_ims2
        t_qs = train_q2
        t_ans = trains_ans2
        
    model.fit([t_ims, t_qs[t_ims.shape[0] * i: t_ims.shape[0]* (i+1)]], t_ans[t_ims.shape[0]*i: t_ims.shape[0]*(i+1)], epochs=1, batch_size=30)

Epoch 1/1
745/745 [==============================] - 184s 247ms/step - loss: 0.9626 - acc: 0.8497
Epoch 1/1
  60/2983 [..............................] - ETA: 11:18 - loss: 6.0414 - acc: 0.0333    

KeyboardInterrupt: 

In [9]:
## Saving the model
# serialize model to JSON
model_json = model.to_json()
with open("model_VQA_4_36.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_VQA_4_36.h5")
print("Saved model to disk")

Saved model to disk


In [9]:
### load the model if saved

from keras.models import model_from_json
import keras
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
from keras import models
from keras import layers
from keras import optimizers

json_file = open("model_VQA_4_36.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# load weights into new model
model.load_weights("model_VQA_4_36.h5")
print("Loaded model from disk")

Using TensorFlow backend.


Loaded model from disk
